# Download ECMWF forecast data from MS Planetary Computer
https://planetarycomputer.microsoft.com/dataset/ecmwf-forecast#overview

## Imports

In [ ]:
import json
import urllib.request
import tempfile

import httpx
import planetary_computer
import pystac_client
from tqdm import tqdm
import xarray as xr

## Query STAC to get data URLs

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
forecast_step = "0h"

In [ ]:
search = catalog.search(
    collections=["ecmwf-forecast"],
    query={
        "ecmwf:stream": {"eq": "enfo"},
        "ecmwf:step": {"eq": forecast_step},
    },
)
items = search.get_all_items()
len(items)

In [ ]:
item = max(items, key=lambda item: item.datetime)
item

In [ ]:
item.properties

In [ ]:
data_url = item.assets["data"].href
index_url = item.assets["index"].href

## Attempt to use byte offsets to avoid downloading whole file
Not working (file loads, but no data in it?)

In [ ]:
# Get byte off
r = httpx.get(index_url)
assert r.status_code == 200
chunks = [json.loads(t) for t in r.text.strip().split("\n")]
ch = [c for c in chunks if c["param"] == "t"][0]
print(ch)

In [ ]:
# offset, length = ch["_offset"], ch["_length"]
# start, end = offset, offset + length
# headers = {"Range": f"bytes={start}-{end}"}
headers = {}

## Download file to /tmp dir

In [ ]:
def download_file(url, headers=None):
    with tempfile.NamedTemporaryFile(delete=False) as file:
        with httpx.stream("GET", data_url, headers=headers) as response:
            total = int(response.headers["Content-Length"])
            with tqdm(total=total, unit_scale=True, unit_divisor=1024, unit="B") as progress:
                num_bytes_downloaded = response.num_bytes_downloaded
                for chunk in response.iter_bytes():
                    file.write(chunk)
                    progress.update(response.num_bytes_downloaded - num_bytes_downloaded)
                    num_bytes_downloaded = response.num_bytes_downloaded
    return file.name

In [ ]:
# NB this will download a ~2.5GB file
file = download_file(data_url, headers=headers)

## Open and explore dataset

In [ ]:
ds = xr.open_dataset(
    file,
    engine="cfgrib",
    filter_by_keys={"dataType": "cf"},
)
ds

In [ ]:
keys = {
    "u10": "wind u",
    "v10": "wind v",
    "t": "air temp",
    # downward flux: missing
    # tp values are 0?
    # precip rate missing
}

In [ ]:
ds.sel(
    latitude=slice(45, 40),
    longitude=slice(10, 15),
    isobaricInhPa=1000,
)[keys.keys()]